# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [48]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
from pprint import pprint

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [49]:
city_data = "../cities_output2.csv"

city_data_df = pd.read_csv(r'../cities_output2.csv')
city_data_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp (F),Wind Speed
0,hermanus,0,ZA,1635109587,71,-34.4187,19.2345,57.90,8.90
1,ribeira grande,5,PT,1635109587,71,38.5167,-28.7000,66.58,8.99
2,tuktoyaktuk,88,CA,1635109587,90,69.4541,-133.0374,26.35,3.24
3,east london,73,ZA,1635109356,65,-33.0153,27.9116,62.17,12.24
4,miles city,1,US,1635109588,47,46.4083,-105.8406,62.51,6.91


In [50]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555 entries, 0 to 554
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   City          555 non-null    object 
 1   Cloudiness    555 non-null    int64  
 2   Country       552 non-null    object 
 3   Date          555 non-null    int64  
 4   Humidity      555 non-null    int64  
 5   Lat           555 non-null    float64
 6   Lng           555 non-null    float64
 7   Max Temp (F)  555 non-null    float64
 8   Wind Speed    555 non-null    float64
dtypes: float64(4), int64(3), object(2)
memory usage: 39.1+ KB


In [51]:
city_data_df['Country'][0]

'ZA'

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [76]:
figure_layout = {'width': '500px', 'margin': '0 auto 0 auto'}
fig = gmaps.figure(layout=figure_layout)

marker_locations = city_data_df[['Lat', 'Lng']]

markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px', margin='0 auto 0 auto', width='500px'))

In [53]:
humidity_list = city_data_df["Humidity"].tolist()


In [70]:
fig = gmaps.figure(layout =figure_layout)

heat_layer = gmaps.heatmap_layer(marker_locations, weights=humidity_list, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px', margin='0 auto 0 auto', width='500px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [55]:
perfect_weather=city_data_df.loc[(city_data_df["Max Temp (F)"]>=75)&(city_data_df["Max Temp (F)"]<=80)&(city_data_df["Humidity"]<=35)&(city_data_df["Wind Speed"]<=10)]

perfect_weather


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp (F),Wind Speed
80,saint george,1,US,1635109561,33,37.1041,-113.5841,75.67,1.01
231,marrakesh,0,MA,1635109669,31,31.6315,-8.0083,75.27,3.44
233,funtua,72,NG,1635109670,27,11.5233,7.3081,77.41,5.88
489,ramotswa,67,BW,1635109767,34,-24.8716,25.8699,76.01,8.66


In [56]:
hotel_df = perfect_weather

In [57]:
hotelname=[]
index=0

for index, row in hotel_df.iterrows():
    params = {
        "radius": 30000,
        "rankby": "prominence",
        "keyword": "hotel",
        "key": g_key}
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel_json = requests.get(url, params=params).json()
    hotelname.append(hotel_json['results'][0]['name'])
hotel_df["Hotel Name"]=hotelname


<ipython-input-57-5006fd2e6165>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]=hotelname


In [58]:
len(hotelname)

4

In [59]:
pprint(hotel_json)

{'html_attributions': [],
 'next_page_token': 'Aap_uEAjzqwDnmo5tHbWrKCutnil60yTY20OJ6p77gSlvHGmsLLZittZIeIUn3i3HIWQPC9PS3ZRjKO7UApzhoUnoSNymEZ8aOvSb5Cg6ZRbNfzb1FZGpCa9Fq4C7h_A2GmvaDbVkRBLrRiCNTe2ht2ELY_FW0gcfyhmzO_hN2WOFUzX0phG6jQTFzU3Aziog9sboT4ywuqHiqFDAcXZKK7PZRlEHytxZRiKWv1sAxVilU9I8A54cRqJv_E4IxOBIeTYz9uz1KxHFSbXQ4pjNvAsHIG8FpqXsJtWuUpQw5W5XdwpYECAgTBFJ6943Hlt8q12WB_Pmtj4d3_hBMJG1PuBczbcDTSzRQ17g-t66t9-5d3SFHR_xA9p07MaoFqFtMLO0PdmGdearwxmwkqzjgI-suRJJmMHjMQ0AI9d2AaC_yPN96mUjgLzE4RY',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -24.6525343, 'lng': 25.90381},
                           'viewport': {'northeast': {'lat': -24.65112962010728,
                                                      'lng': 25.90499127989272},
                                        'southwest': {'lat': -24.65382927989272,
                                                      'lng': 25.90229162010728}}},
              'icon': 'https://maps.gstatic.com/mapfil

In [60]:
print(hotelname)

['Red Lion Hotel & Conference Center St. George', 'La Mamounia', 'Jamil Hotel', 'Protea Hotel by Marriott Gaborone Masa Square']


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [61]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [62]:
hotel_info

['\n<dl>\n<dt>Name</dt><dd>Red Lion Hotel & Conference Center St. George</dd>\n<dt>City</dt><dd>saint george</dd>\n<dt>Country</dt><dd>US</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>La Mamounia</dd>\n<dt>City</dt><dd>marrakesh</dd>\n<dt>Country</dt><dd>MA</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Jamil Hotel</dd>\n<dt>City</dt><dd>funtua</dd>\n<dt>Country</dt><dd>NG</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Protea Hotel by Marriott Gaborone Masa Square</dd>\n<dt>City</dt><dd>ramotswa</dd>\n<dt>Country</dt><dd>BW</dd>\n</dl>\n']

In [72]:
# Add marker layer ontop of heat map
info_box_content = hotel_info
markers = gmaps.marker_layer(locations, hotel_info)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px', margin='0 auto 0 auto', width='500px'))